# App Store and Google Play profitable App profiling

Working for a company that builds Android and iOS mobile apps that are available on the Google Play and App store, for this project we are going to be comparing App profiles. The solutions will help developers make better decisions on how to design and update their apps.

The apps designed by the company are all free to download and install so all revenue comes from Ads. As revenue for adverts comes from in app views of the advert we are going to be comparing frequency data so developers can identify which apps are more attractive to their customer base.


## Collecting the data

As of 2018 there was roughly around 2 million Apps in both Google Play and the App Store.

Collecting data on 4+ million apps will take a long time and costly to attain the data in the first place. To avoid suffering a large personal cost and in the respect of time we are going to locate some relevant sample data for free.

Fortunately there is two data sets already fit for this purpose:

- [Google Play data](https://www.kaggle.com/lava18/google-play-store-apps/home) containing data about approximately ten thousand Android apps from Google Play
- [App Store data](https://www.kaggle.com/ramamet4/app-store-apple-data-set-10k-apps/home) containing data about approximately seven thousand iOS apps from the App Store

First off we have to open the two datasets and read them in.
*(Note: we are working with csv's here so import the reader from the csv module. We also set two variables for the data content and header)*



In [1]:
# Import the reader from the csv module
from csv import reader

# Google Play data imported
open_google_file = open("googleplaystore.csv")
read_google_file = reader(open_google_file)
google_data = list(read_google_file)
google_data_header = google_data[0]
google_data_content = google_data[1:]

# App Store data imported
open_apple_file = open("AppleStore.csv")
read_apple_file = reader(open_apple_file)
apple_data = list(read_apple_file)
apple_data_header = apple_data[0]
apple_data_content = apple_data[1:]


FileNotFoundError: [Errno 2] No such file or directory: 'googleplaystore.csv'

To explore the two datasets we will create a function called `explore_data()` that will allow us to easily read the rows and columns of a data set. An optional variable will be added to allow the calculation of the number of rows and columns in a dataset.

In [ ]:
def explore_data(dataset, indexStart, indexEnd, count_rows_columns=False):
    rows = dataset[indexStart:indexEnd]
    for row in rows:
        print(row)
        # insert blank line after each row list
        print("\n")
        
    if count_rows_columns is True:
        print("Number of rows: ", len(dataset))
        print("Number of columns: ", len(dataset[0]))
        

Let's now test the test the Google data and print the first 3 rows. we will also initialise the counting section of the function and have a look at the header of row of the Google data.

In [ ]:
# show the Google header column

print(google_data_header)


At first look, for frequency testing we will most likely need the columns *"App", "Category", "Reviews", "Installs", "Type", "Price" and "Genres"*. Now we know the header works let's test our function

In [ ]:
# Initialise the function with google data

explore_data(google_data_content, 0, 3, True)


***
Success! The function works and it seems the Google data has **13 columns** and **10841 rows**. Let's do the same with the Apple data and see what we get.

In [ ]:
# Show the header data for Apple

print(apple_data_header)
print("\n")
explore_data(apple_data_content, 0, 3, True)


***
So this data set contains 7197 iOS apps. Similar to Google we will detail the columns that might of most use; *"track_name", "currency", "price", "rating_count_tot", "rating_count_ver", and 'prime_genre'.*

If you are unsure on what any of the column headers show there are descriptions that can be found in the documention:

[Documentation](https://www.kaggle.com/ramamet4/app-store-apple-data-set-10k-apps/home)


# Data cleansing

## Removing incorrect rows

The Google data set has a dedicated discussion section, and we can see that one of the discussions outlines an error for row 10472.

Upon review it seems there seems to be a missing column for this row. To confirm this and also check there are no other rows that are also  exhibiting the same problem we are going to search the Google data for any rows that are not the same length of the header and print the index of that row


In [ ]:
"""
We are going to use our data set without the header so we
need to bring in the header row as a variable, this is just
a personal preference
""" 

for row in google_data_content:
    row_length = google_data_header
    if len(row) != len(row_length):
        print(row)
        print("\n")
        print(google_data_content.index(row))
        


It seems there is only one row that exhibits this problem and its on row 10472 of the data set for the app "Life Made WI-Fi Touchscreen Photo Fram". *Remember if we include the header row it would be one index more at 10473*

To check what the problem is lets print the header list alongside the troublesome row and see if anything stands out.


In [ ]:
print(google_data_header)
print("\n")
print(google_data_content[10472])

If we look at the third element of both rows we can see that the Life Made app seems to have a rating of 19. This is clearly a mistake as Google Play apps have a max rating of 5. We are therefore going to delete the app.

In [ ]:
# We simply delete the row to remove the app

del google_data_content[10472]

# If we now print the row again you can see it has changed

print(google_data_content[10472])

***

## Removing duplicate entries

### Identifying duplicate entries

If we continue to explore the discussions for the Google data at; [Discussions](https://www.kaggle.com/lava18/google-play-store-apps/discussion)

You will see on a couple of occurances there is mention of duplicate entries for certains apps. We therefore need to identify this apps, work out which entries to keep and delete the extra.

First things first we can identify duplicate entries by tracking down rows that have the same app name


In [ ]:
"""
Set two lists for duplicate entries and 
unique entries to be saved to
"""

duplicate_entries = []
unique_entries = []

for row in google_data_content:
    app_name = row[0]
    if app_name in unique_entries:
        duplicate_entries.append(app_name)
    else:
        unique_entries.append(app_name)

print("Unique entries: ", len(unique_entries))
print("\n")
print("Duplicate entries: ", len(duplicate_entries))
print("\n")

# Creating total_entries is just for tidyness
total_entries = len(duplicate_entries) + len(unique_entries)

# A quick sense check to make sure we checked every row
if total_entries == len(google_data_content):
    print("All rows have been assigned to duplicate or unique")
else:
    print("Mistakes were made")



Now that we have a list of all the duplicate app names we can check a few to test.


In [ ]:
# look at the first 20 duplicate entries

print(duplicate_entries[:20])



Using a small piece of code we can create a function that will take an index for the `duplicate_entries` list and return us all the rows with that app_name


In [ ]:
def dup_check(dup_index):
    for row in google_data_content:
        dup_name = row[0]
        if dup_name == duplicate_entries[dup_index]:
            print(row)
            print("\n")

As we are looking at Google data lets look at some duplicates for Google apps and see if we can see what's going on. from our first 20 lets look at the following:

- "Google My Buiness" = index(2)
- "Google Ads" = index(7)
- "Google Analytics" = index(18)

As we built a function we can just run it three times and see what we get


In [ ]:
dup_check(2)
dup_check(7)
dup_check(18)

If we look through the above list of lists we can see the only number that changes is the "number of reviews" in column 4 for the "Google Ads" app.

We know we cannot keep multiple entries for the same app  and ideally we would not want to delete rows randomly. By noticing the change in column 4 it is safe to say that the duplicate rows with the highest reviews is the latest entry and therefore the app row that we are going to keep.

In the next section we will look at how we might go about deleting the duplicate apps to meet this criteria

### Deleting apps based on criteria

There are two steps to deleting the duplicate rows:

- Create a dictionary where each key is a unique app name, and the value is the highest number of reviews of that app
- Use the dictionary to create a new data set, which will have only one entry per app (and we only select the apps with the highest number of reviews)

#### Step one:

As this can get a bit loopy (literally in this case) it's best broken down into a series of steps:

- Assign an empty dictionary
- loop over the Google data looking at every row
- Assign a variable for the app name and number of reviews
- the logic then goes as follows:
  - If the app name isn't in the dictionary add it and the corresponding number of reviews.
  - If the app name is in the dictionary check if the new app entry has a higher review number than the value in the dictionary. If it does replace the app name dictionary key with the new number of reviews value
 

In [ ]:
reviews_max = {}

for row in google_data_content:
    name = row[0]
    n_reviews = float(row[3])
    
    if name in reviews_max and reviews_max[name] < n_reviews:
        reviews_max[name] = n_reviews
        
    elif name not in reviews_max:
        reviews_max[name] = n_reviews

We know from earlier on in our code the number of unique entries should be **9659**. As reviews_max should now contain a dictionary of unique values we can check that this equals the length of the list of unique values from earlier. 

In [ ]:
print("Oringinal unique entries list: ", len(unique_entries))
print("\n")
print("length of reviews_max dictionary: ", len(reviews_max))
print("\n")

if len(reviews_max) == len(unique_entries):
    print("Excellent, they match!")
else:
    print("Mistakes were made")


We now have a nice dictionary full of the unique app names and their highest rating. Now for the second part, stripping the original data of all entries that do not exhibit these two variables.


#### Step two:

To strip the data from our main source we will undergo another series of steps when again is best explained in simple English:

- We start by initializing two empty lists, google_data_clean and already_added.
- We loop through the Google data set, and for every iteration:
  - We set the name of the app and the number of reviews to a variable.
  - We then add the current row (app) to the google_data_clean list, and the app name (name) to the already_cleaned list if:
    - The number of reviews of the current app matches the number of reviews of that app as described in the reviews_max dictionary; and
    - The name of the app is not already in the already_added list. We need to add this extra condition to account for those cases where the highest number of reviews of a duplicate app is the same for more than one entry (consider the other Google apps we looked at earlier where the number of reviews was the same for multiple entries). If we just check for reviews_max[name] == n_reviews, we'll still end up with duplicate entries for some apps.

In [ ]:
# This list will be our new cleansed list
google_data_clean = []
# List for dealing with identical rows
already_added = []

for row in google_data_content:
    name = row[0]
    n_reviews = float(row[3])
    if (reviews_max[name] == n_reviews) and (name not in 
                                             already_added):
        google_data_clean.append(row)
        already_added.append(name)
        

We can use the `explore_data` function we created earlier to count the number of rows in our new data set to make sure it is definately **9659**.

In [ ]:
explore_data(google_data_clean, 0, 3, True)

Perfect! That's the correct number of rows.

After going through the data a bit more as we are working for an English based company the next stage is to remove foreign apps. We will explore that now.

## Removing foreign apps

### Identifying non ASCII characters

As some apps are not written using the English language language and we are not interested in keeping these kind of apps, we'll remove them. One way to go about this is to remove each app whose name contains a symbol that is not commonly used in English text — English text usually includes letters from the English alphabet, numbers composed of digits from 0 to 9, punctuation marks (., !, ?, ;, etc.), and other symbols (+, *, /, etc.).

All these characters that are specific to English texts are encoded using the ASCII standard. Each ASCII character has a corresponding number between 0 and 127 associated with it, and we can take advantage of that to build a function that checks an app name and tells us whether it contains non-ASCII characters.

So a simple initial test we can do is elimate anything outside of the 0 - 127 range. Below we have created a function to test any app in any dataset based on its index.

In [ ]:
def english_index_check(dataset, app_index):
    app = dataset[app_index]
    app_name = str(app[0])
    print(app_name)
    for character in app_name:
        if ord(character) > 127:
            return False
    return True
            
            
english_index_check(google_data_clean, 256)

As you can see for the app in index position 256 of our Google data there is non English text and therefore the funtion returns a False. We can check another to prove it works for English text.

In [ ]:
english_index_check(google_data_clean, 258)

So for plain English text we get a True as expected. Unfortunately this method creates a problem. Some apps use symbols, emojis and characters that fall outside of the ASCII range. Because of this, we'll remove useful apps if we use the function in its current form. As example can be seen below.

In [ ]:
english_index_check(google_data_clean, 257)

The app is clearly English text but the use of the music symbols at the end returns a False. We therefore need to find a way around this.

### A simple solution

As there is rarely apps where there is lots of use of emojis and symbols a potential solution is to only label an app as non english when it has more than three characters that fall outside of the standard ASCII range. We can simply edit our previous function to achieve this.

In [ ]:
def english_index_check(dataset, app_index):
    app = dataset[app_index]
    app_name = str(app[0])
    print(app_name)
    non_character = 0
    for character in app_name:
        if ord(character) > 127:
            non_character += 1
            
    if non_character > 3:
        return False
    else:
        return True

    
english_index_check(google_data_clean, 257)

It is definately the case that this new system will let some non English apps through, but it will also now let a larger proportion of English apps through as well. You can see this in the example above as the the music symbols no longer identify the app as non English.

Now we have our system we need to edit the function to be able to take in an app  name as a string instead of the index of the app and we can loop over both datasets.

In [ ]:
def english_string_check(app_name):
    count_non_ascii = 0
    
    for letter in app_name:
        if ord(letter) > 127:
            count_non_ascii += 1
    
    if count_non_ascii > 3:
        return False
    else:
        return True

Let's run a quick check to make sure it works

In [ ]:
print(english_string_check("Docs To Go™ Free Office Suite"))
print(english_string_check("Instachat 😜"))
print(english_string_check("爱奇艺PPS -《欢乐颂2》电视剧热播"))

These answers are as expected so let's run this for everything in our cleaned Google data and Apple data.

In [ ]:
google_english = []
apple_english = []

for app in google_data_clean:
    app_name = app[0]
    if english_string_check(app_name) is True: 
        # 'is True' added for personal preference. It is not required
        google_english.append(app)
        
for ios_app in apple_data_content:
    # Remember the Apple data's first column was ID not the app name
    ios_app_name = ios_app[1]
    if english_string_check(ios_app_name) is True:
        apple_english.append(ios_app)

Again we can use the `explore_data` function to check these the english app checks worked as intended.

In [ ]:
explore_data(google_english, 0, 3, True)
print('\n')
explore_data(apple_english, 0, 3, True)

A the number of apps has reduced and the first few columns show no obvious issues for this level of study we can consider this a success and move on.

## Removing non free apps

As we mentioned at the beginning of this document we are only interested in apps that are free to both download and install. So far we have removed:

- Incorrect data
- Duplicate entries
- Non enlgish apps

Removing the non free apps from our data will be the last stage before we get stuck into some analysis.

Luckily we have a price element for both datasets we can easily access. Using a similar layout for the previous stage we can easily filter the data.

In [ ]:
google_cleaned = []
apple_cleaned = []

for app in google_english:
    price = app[7]
    if price == "0":
        google_cleaned.append(app)
        
for ios_app in apple_english:
    cost = ios_app[4]
    if cost == "0.0":  # Remember Apple data price is stored as a float
        apple_cleaned.append(ios_app)

# Explore the datasets one final time        
explore_data(google_cleaned, 0, 3, True)
print('\n')
explore_data(apple_cleaned, 0, 3, True)

Finally we are left with 8864 Google apps and 3222 Apple apps, which is plenty to do some analysis on.

## Most Common Apps by Genre

### Collecting the data

As our revenue is based on ads and therefore the number of views that we get following download and installs of our apps. We are going to look at the most popular types of apps that are more likely to attract future customers.

The current stategy for app builds is as follows:

- Build a minimal Android version of the app, and add it to Google Play.
- If the app has a good response from users, we then develop it further.
- If the app is profitable after six months, we also build an iOS version of the app and add it to the App Store.

As our final product will be released on both android and ios systems we need to analyse app profiles from both sets of data to make sure the information we learn about one platform stays true on the other.

We are going to start by looking at the most common genres for apps in the App store and Google Play. Using the methods we developed earlier we are going to first build a frequency table for the prime_genre column of the App Store data set, and the Genres and Category columns of the Google Play data set.

#### Frequency table function

This function is used to generate the frequency tables for whatever column we wish to look at. It displays the frequencies as a percentage for the final output.

In [ ]:
def freq_table(data, index):
    frequency_table = {}
    percentage_table = {}
    total = 0
    
    for row in data:
        total += 1
        item = row[index]
        
        if item in frequency_table:
            frequency_table[item] += 1
        else:
            frequency_table[item] = 1
        
    for key in frequency_table:
        percentage = (frequency_table[key] / total) * 100
        percentage_table[key] = percentage
    
    return percentage_table

#### Sorting function

The sorting function takes the percentages for each element of each column discovered in the frequency function and sorts them from highest to lowest.

In [2]:
def ordered_table(data, index):
    percent_table = freq_table(data, index)
    display_list = []
    
    for key in percent_table:
        key_val_tuple = (percent_table[key], key)
        display_list.append(key_val_tuple)
    
    table_sorted = sorted(display_list, reverse=True)
    for entry in table_sorted:
        print(entry[1], ":", entry[0])


### Analysing the outputs

Using the both functions we start by looking at the prime_genre column from the App Store dataset.

In [3]:
ordered_table(apple_cleaned, -5)

NameError: name 'apple_cleaned' is not defined

From the data we can see that from the filtered dataset showing free and English apps that over half are games (~58%). The other two highest are entertainment and photo/video apps. The rest of the genres make up less than 4% for each respective genre.

From this it is clear that the vast amount of apps on the App store are past time apps for fun. However we are only counting the number of apps at this point so the quantity of apps available may not represent the actual use of the apps.

Let's now look at the Genres and Category columns of the Google Play data set.

In [ ]:
ordered_table(google_cleaned, 1)  # Category

The category column for Google Play tells us a very different story, there are not that many apps designed for fun, and it seems that a good number of apps are designed for practical purposes (family, tools, business, lifestyle, productivity, etc.).

However, if we investigate this further, we can see that the family category (which accounts for almost 19% of the apps) means mostly games for kids.

Even so, practical apps seem to have a better representation on Google Play compared to App Store. This picture is also confirmed by the frequency table we see for the Genres column:

In [ ]:
ordered_table(google_cleaned, -4)  # Genre

The actual difference between the Genre and Category column isn't actually explained fully. One thing we can see is that the Genre column has a lot more detail than the Category. We're only looking for the bigger picture at the moment, so we'll only work with the Category column moving forward.

As we mentioned before the frequency may be deceptive however. So to try and get a better understanding of what apps are most successful by genre we will explore the number of users of the apps.

## Most popular apps by genre

### App store


One way to find out what genres are the most popular (have the most users) is to calculate the average number of installs for each app genre. For the Google Play data set, we can find this information in the Installs column, but for the App Store data set this information is missing. As a workaround, we'll take the total number of user ratings as a basic replacement, which we can find in the rating_count_tot column.

Below, we calculate the average number of user ratings per app genre on the App Store:

In [ ]:
genre_table = freq_table(apple_cleaned, -5)

for genre in genre_table:
    total = 0
    len_genre = 0
    
    for row in apple_cleaned:
        genre_app = row[-5]
        if genre_app == genre:
            user_rating = float(row[5])
            total += user_rating
            len_genre += 1
    
    avg_users = total / len_genre
    print(genre, ":", avg_users)


On average navigation is the highest so we will explore that first

In [ ]:
for apps in apple_cleaned:
    if apps[-5] == "Navigation":
        print(apps[1], ":", apps[5])  # name and ratings

As we can see this is almost entirely dominated by Google and Waze so is not a good choice. Equally we can assume the same for other higher rankers that have a few dedicated pieces of software. A prime example of this is social networking which will be heavily skewed by the likes of Facebook, Twitter, Instagram etc.

In [ ]:
for apps in apple_cleaned:
    if apps[-5] == "Music":
        print(apps[1], ":", apps[5])

As we are looking at a primarily fun market as our basis for apps. The karaoke, music editing and streaming sections of music seem to have a reasonable average so there could be a market here for a new app which a high view rate for our ads. Also due to the length of time music editing or streaming takes to enjoy the content the exposure to the ads will be lengthy.

The only downside to this is there is already a lot of similar ads for this market so a new strategy of how to make this work will need to be devised. Something along the lines of combining streaming recent music that can then be edited and sung along to after combines all the elements that have high user counts, but will also keep them entertained long enough for ad benefits.

### Google Play

For the Google Play market, we actually have data about the number of installs, so we should be able to get a clearer picture about genre popularity. However, the install numbers don't seem precise enough — we can see that most values are open-ended (100+, 1,000+, 5,000+, etc.):

In [ ]:
ordered_table(google_cleaned, 5) 

To perform calculations on this data set we need the installs to be a number not a string so we will assume that the `"+"` is unimportant and the number represents the exact number of installs. We will also need to remove the comma so it can be recognised as a float.

In [ ]:
google_play = freq_table(google_cleaned, 1)  # category index is 1

for category in google_play:
    total = 0  # total will store the sum of the installs
    len_category = 0  # this variable will store the number of specific apps per genre
    for app in google_cleaned:
        category_app = app[1]
        if category_app == category:
            n_installs = app[5]
            n_installs = n_installs.replace("+", "")
            n_installs = n_installs.replace(",", "")
            total += float(n_installs)
            len_category += 1
    avg_installs = total / len_category
    print(category, ":", avg_installs)
    

On average, communication apps have the most installs: 38,456,119. We can assume this data is skewed as before by a few key players (WhatsApp, Facebook Messenger, Skype, Google Chrome, etc).. We can confirm this quickly to prove our point:

In [ ]:
for app in google_cleaned:
    if app[1] == 'COMMUNICATION' and (app[5] == '1,000,000,000+'
                                      or app[5] == '500,000,000+'
                                      or app[5] == '100,000,000+'):
        print(app[0], ':', app[5])

As expected the big players are dominating. Again we can see this elsewhere in the other categories: the video players category, which is the runner-up with 24,727,872 installs. The market is dominated by apps like Youtube, Google Play Movies & TV, or MX Player. The pattern is repeated for social apps (where we have giants like Facebook, Instagram, Google+, etc.), photography apps (Google Photos and other popular photo editors), or productivity apps (Microsoft Word, Dropbox, Google Calendar, Evernote, etc.).

Another area worth exploring in this case is books and references. With roughly 8 million views it has a high average.

In [ ]:
for app in google_cleaned:
    if app[1] == 'BOOKS_AND_REFERENCE':
        print(app[0], ':', app[5])

The book and reference genre includes a large number of apps: software for processing and reading ebooks, various collections of libraries, dictionaries, tutorials on programming or languages, etc. It seems there's still a small number of extremely popular apps that skew the average though:

In [ ]:
for app in google_cleaned:
    if app[1] == 'BOOKS_AND_REFERENCE' and (app[5] == '1,000,000,000+'
                                            or app[5] == '500,000,000+'
                                            or app[5] == '100,000,000+'):
        print(app[0], ':', app[5])

However, it looks like there are only a few very popular apps, so this market still shows potential. Let's try to get some app ideas based on the kind of apps that are somewhere in the middle in terms of popularity (between 1,000,000 and 100,000,000 downloads):

In [ ]:
for app in google_cleaned:
    if app[1] == 'BOOKS_AND_REFERENCE' and (app[5] == '1,000,000+'
                                            or app[5] == '5,000,000+'
                                            or app[5] == '10,000,000+'
                                            or app[5] == '50,000,000+'):
        print(app[0], ':', app[5])

This area seems to be dominated by software reading ebooks, as well as various collections of libraries and dictionaries, so it's probably not a good idea to build similar apps since there'll be some significant competition. Not to mention as they are free it will take a while to build a portfolio of books and/or gather enough information for a library resource.

We also notice there are quite a few apps built around the book Quran in this set, we also know from the App Store the Bible has a large number of views, which suggests that building an app around a popular book can be profitable. It seems that taking a popular book (perhaps a more recent book) and turning it into an app could be profitable for both the Google Play and the App Store markets.

However, it looks like the market is already full of recent books, so we need to add some special features besides the raw version of the book. This might include daily quotes from the book, an audio version of the book, quizzes on the book, a forum where people can discuss the book, etc.

## Final thoughts

In this project, we analyzed data about the App Store and Google Play mobile apps with the goal of recommending an app profile that can be profitable for both markets.

We concluded that taking a popular book (perhaps a more recent book) and turning it into an app could be profitable for both the Google Play and the App Store markets.

We also identified a music app that has a new fun twist on the current examples of purely game or streaming apps would do well in the App Store market. There was no specific area to check this in Google Play without doing a lot more research, this could perhaps be something that is explored in the future.